<a href="https://colab.research.google.com/github/imsurgeon/dls-mipt/blob/master/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F%20%D0%A1%D0%B8%D0%BC%D0%BF%D1%81%D0%BE%D0%BD%D0%BE%D0%B2/simpsons_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Екатерина Апраксина
160077195

Kaggle score: 0.97874

Импорты

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import numpy as np
from sklearn.model_selection import train_test_split
import os
import natsort
from PIL import Image
import copy
import pandas as pd
from sklearn.metrics import f1_score
torch.manual_seed(42)
np.random.seed(42)

GPU

In [2]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

Using CPU


In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =  True 
torch.backends.cudnn.deterministic = False

**Data**

Преобразования

In [4]:
imshow = lambda x: plt.imshow(x.permute(1,2,0))

In [5]:
transforms_dict = {
    'eval': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ]),
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.8, 1), ratio=(0.75, 1.333333), interpolation=2),
        torchvision.transforms.RandomPerspective(distortion_scale=0.3, p=0.9, interpolation=3, fill=0),
        torchvision.transforms.RandomAffine(degrees=30, shear=20, resample=False),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

Загрузка датасетов

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [7]:
train_path = '/content/gdrive/My Drive/simpsons/data/train/'
test_path = '/content/gdrive/My Drive/simpsons/data/test/'

In [8]:
data = datasets.ImageFolder(root = train_path, transform = transforms_dict['eval'])
data_augmented = datasets.ImageFolder(root = train_path, transform = transforms_dict['train'])

n_classes = len(data.classes)

label_to_name = {v: k for k, v in data.class_to_idx.items()}

train_indices, val_indices, _, _, = train_test_split(np.arange(len(data)), data.targets, test_size = 0.1, stratify=data.targets, random_state = 42)

train_dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(data_augmented, train_indices), batch_size = 100)
val_dataloader = torch.utils.data.DataLoader(torch.utils.data.Subset(data, val_indices), batch_size = 100)
final_train_dataloader = torch.utils.data.DataLoader(data_augmented, batch_size=100, shuffle=True)

Test

In [9]:
class TestImageFolder(torch.utils.data.Dataset):

    def __init__(self, root, transform, ext = '.jpg'):
        self.root = root
        self.transform = transform
        files_unsorted = [i for i in os.listdir(root) if ext in i]
        self.files = natsort.natsorted(files_unsorted) 

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        paths = os.path.join(self.root, self.files[idx])
        image = Image.open(paths).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

In [10]:
test_data = TestImageFolder(test_path, transforms_dict['eval'], ext='.jpg')
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=100)

Цикл обучения

In [11]:
best_f1 = 0
best_model = None

In [12]:
def train(model, train_dataloader, val_dataloader, num_epoch, loss_function, optimizer, scheduler, device):

    global best_f1
    global best_model

    start_time = time.time()
    
    train_losses = []
    val_losses = []
    val_f1 = []
    
    train_size = float(len(train_dataloader.dataset))
    if val_dataloader is not None:
        val_size = float(len(val_dataloader.dataset))
    
    for i in range(num_epoch):
        
        print('epoch',i)

        train_running_loss = 0
        model.train(True)
        for j, (inputs, labels) in enumerate(train_dataloader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_running_loss += loss.item() * inputs.size(0)

        train_losses.append(train_running_loss/train_size)
          
        if val_dataloader is not None:
            val_preds = []
            val_true = []
            val_running_loss = 0
            model.train(False)
            with torch.no_grad():
                for inputs, labels in val_dataloader:

                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    outputs = model(inputs)
                    loss = loss_function(outputs, labels)

                    val_running_loss += loss.item() * inputs.size(0)
                    _, batch_preds = torch.max(outputs, axis=1)
                    val_preds += batch_preds.tolist()
                    val_true += labels.tolist()
            
            val_losses.append(val_running_loss/val_size)
            val_f1.append(f1_score(val_true, val_preds, average='macro'))
            
            print('Validation loss:', val_losses[-1])
            print('Validation f1:', val_f1[-1])
        
            if val_f1[-1] > best_f1:
                best_f1 = val_f1[-1]
                best_model = copy.deepcopy(model)       
        
        if scheduler is not None:
            scheduler.step()
        
    end_time = time.time()
    
    print('Total time: ',end_time - start_time)
    print('Average time per epoch: ',(end_time - start_time) / num_epoch)
    
    return {'train_losses': train_losses,
            'val_losses': val_losses,
            'val_f1': val_f1,
           }

Model

In [13]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

In [14]:
for param in model.parameters():
    param.requires_grad = False
    
for param in model.layer4.parameters():
    param.requires_grad = True
    
model.fc = torch.nn.Linear(num_ftrs, n_classes)

model = model.to(device)

In [15]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3)

In [16]:
result_dict_frozen = train(model, final_train_dataloader, None, 5, loss_function, optimizer, scheduler, device)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
Total time:  18912.821585655212
Average time per epoch:  3782.5643171310426


In [17]:
for param in model.parameters():
    param.requires_grad = True

In [18]:
result_dict_unfrozen = train(model, final_train_dataloader, None, 5, loss_function, optimizer, scheduler, device)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
Total time:  19552.374881982803
Average time per epoch:  3910.4749763965606


**Предсказание**

In [19]:
model = model.to(device)
model.train(False)
with torch.no_grad():
    test_preds = []
    for batch in test_dataloader:
        batch = batch.to(device)
        _, preds = model(batch).max(axis=1)
        test_preds.append(preds.tolist())
    test_preds = sum(test_preds,[])
test_preds = [label_to_name[label] for label in test_preds]
submission_df = pd.DataFrame([*zip(test_data.files,test_preds,)]).rename(columns = {0: 'Id', 1: 'Expected'})

In [20]:
submission_df.to_csv('submission.csv',index=False)